In [ ]:
import pickle
import plyfile
import numpy as np
import open3d as o3d
import h5py
import numpy as np
import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT' #adjust this to your path
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r, od3du_utils

#adjust the following paths
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
files_dir = '/local/home/ekoller/R3Scan/files'

new_scan_id =   "74ef846e-9dce-2d66-83d5-294aac7b1b0f" #take whatever test set rescan scan ID you want
curr_scan_id = scan3r.get_reference_id(data_dir,new_scan_id) # is the reference scene
print(curr_scan_id)



In [ ]:
#access the predicted reference objects from the final prediction
def acces_predicted_objects(scan_id):
    all_centers = {}
    filename = osp.join(files_dir, "Predicted_Centers", scan_id + ".h5")
    with h5py.File(filename, 'r') as h5file:
        for obj_id in h5file.keys():
            obj_group = h5file[obj_id]
            center = np.array(obj_group['center'])
            points = np.array(obj_group['points'])
            votes = int(obj_group['votes'][()])
            
            # Add it back to the dictionary
            all_centers[int(obj_id)] = {
                'center': center,
                'points': points,
                'votes': votes
            }
    return all_centers

this segment displays the predicted pointcloud and the ground truth point cloud/ mesh

In [ ]:
# create the newly predicted pointcloud
all_centers = acces_predicted_objects( new_scan_id)
#initialize the final pointcloud
predicted_point_cloud = o3d.geometry.PointCloud()

#colour
#for the rescan
obj_colors = od3du_utils.get_id_colours(data_dir, new_scan_id)
#for the reference scan
obj_colors_ref = od3du_utils.get_id_colours(data_dir, curr_scan_id)

# Iterate over all objects in predicted 3D centers: all_centers
for obj_id, data in all_centers.items():

        if obj_id not in obj_colors:
            color = np.array(obj_colors_ref[obj_id], dtype=np.float32)
            color = np.reshape(color, (3,))
            color = color / 255.0

        else: 
                
            points = data['points']  # Get points
            color = np.array(obj_colors[obj_id], dtype=np.float32)
            color = np.reshape(color, (3,))
            color = color / 255.0

        # Create a point cloud for the current object's points
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)

        # Create a color array of the same size as the number of points, and assign the same color to all points
        point_cloud.colors = o3d.utility.Vector3dVector(np.tile(color, (len(points), 1)))

        # Add the current object's point cloud to the final point cloud
        predicted_point_cloud += point_cloud

        center = data["center"]  # Get the center point
        # Create a sphere mesh centered at `data["center"]`
        sphere_radius = 0.15  # Adjust the radius of the sphere
        sphere_mesh = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
        sphere_mesh.translate(center)  # Move the sphere to the center point

        # Sample points from the mesh surface to create a point cloud
        sphere_pcd = sphere_mesh.sample_points_poisson_disk(number_of_points=500)  # Adjust number of points if needed

        # Set a unique color for the sphere (e.g., red)

        sphere_pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (len(sphere_pcd.points), 1)))

        # Add the sphere point cloud to the final point cloud
        predicted_point_cloud += sphere_pcd





#also access the ground truth rescan mesh to be able to compare predicted vs ground truth objects
pathToMesh = osp.join(data_dir,"scenes", new_scan_id, "labels.instances.align.annotated.v2.ply")
rescan_gt_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#make sure the colours are a scaled accordingly
colors = np.asarray(rescan_gt_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(rescan_gt_mesh.vertex_colors)
rescan_gt_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)





#also access the reference mesh to be able to spot movements inbetween the rescan and reference scan
pathToMesh = osp.join(data_dir,"scenes", curr_scan_id, "labels.instances.align.annotated.v2.ply")
reference_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#make sure the colours are a scaled accordingly
colors = np.asarray(reference_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(reference_mesh.vertex_colors)
reference_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)



"""  Choose what you want to visualize!!!
"""

# Visualize both together using Open3D: predicted_point_cloud (predicted with OD3DU), rescan_gt_mesh (the ground truth rescan mesh), reference_mesh ( the mesh of the reference scene)
o3d.visualization.draw_geometries([predicted_point_cloud, rescan_gt_mesh], window_name="Point Clouds and Meshes")





